In [1]:
import pandas as pd
import requests
from datetime import datetime

# Caminho do arquivo CSV atualizado
file_path = 'BCB_metadata_atualizado.csv'
output_file = 'BCB_metadata_atualizado.csv'

# Carregar o arquivo de metadados já preenchido
metadata_df = pd.read_csv(file_path, sep=',', quotechar='"', on_bad_lines='skip')

# Renomear as colunas em português, caso necessário
metadata_df.rename(columns={
    'code': 'código',
    'name': 'nome',
    'description': 'descrição',
    'refreshed_at': 'atualizado_em'
}, inplace=True, errors='ignore')

# Função para separar "Units: " da descrição
def extrair_unidade(description):
    if "Units: " in description:
        unidade = description.split("Units: ")[1].strip()
        descricao_atualizada = description.split("Units: ")[0].strip()
        return descricao_atualizada, unidade
    return description, None

# Função para consultar a série na API do Banco Central
def baixar_dados_serie(codigo):
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=json'
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verifica se houve algum erro de resposta HTTP
        dados = response.json()
        return dados, "Sucesso"
    except requests.exceptions.RequestException as e:
        return None, f"Erro de requisição: {e}"
    except ValueError:
        return None, "Erro ao decodificar JSON"
    except Exception as e:
        return None, f"Erro desconhecido: {e}"

# Função para calcular a frequência com base nas datas
def calcular_frequencia(primeira_data, ultima_data, num_registros):
    delta = (ultima_data - primeira_data).days
    if delta == 0:
        return None, "Indeterminado"
    
    media_frequencia = delta / num_registros
    
    if media_frequencia < 10:
        return media_frequencia, "Diária"
    elif media_frequencia < 20:
        return media_frequencia, "Quinzenal"
    elif media_frequencia < 40:
        return media_frequencia, "Mensal"
    else:
        return media_frequencia, "Indeterminado"

# Função para verificar se a série está ativa ou inativa
def verificar_atividade(ultima_data):
    hoje = datetime.today()
    delta_meses = (hoje.year - ultima_data.year) * 12 + hoje.month - ultima_data.month
    return "Ativa" if delta_meses < 2 else "Inativa"

# Função para atualizar os metadados
def atualizar_metadados(codigo_serie):
    try:
        dados, status_download = baixar_dados_serie(codigo_serie)
        if not dados:
            return None, status_download
        
        # Extrair informações relevantes
        df = pd.DataFrame(dados)
        df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
        df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
        
        primeira_data = df['data'].min()
        ultima_data = df['data'].max()
        num_registros = df.shape[0]
        valor_min = df['valor'].min()
        valor_max = df['valor'].max()
        media_valor = df['valor'].mean()
        desvio_padrao_valor = df['valor'].std()

        # Calcular a frequência e verificar se a série está ativa
        media_frequencia, frequencia = calcular_frequencia(primeira_data, ultima_data, num_registros)
        status_atividade = verificar_atividade(ultima_data)
        
        # Calcular intervalo em dias após a última data
        intervalo_dias = (datetime.today() - ultima_data).days

        return {
            'num_registros': num_registros,
            'primeira_data': primeira_data,
            'ultima_data': ultima_data,
            'intervalo_dias': intervalo_dias,
            'valor_min': valor_min,
            'valor_max': valor_max,
            'media_valor': media_valor,
            'desvio_padrao_valor': desvio_padrao_valor,
            'media_frequencia': media_frequencia,
            'frequencia': frequencia,
            'status_atividade': status_atividade,
            'status_download': status_download
        }, status_download
    except Exception as e:
        return None, f"Erro durante o processamento: {e}"

# Verificar e atualizar apenas os registros que estão incompletos
for index, row in metadata_df.iterrows():
    # Verificar se o campo 'num_registros' está preenchido, caso contrário, atualizar a série
    if pd.isna(row['num_registros']) or pd.isna(row['ultima_data']):
        codigo_serie = row['código']
        
        # Separar "Unidade" da descrição, se ainda não tiver sido separada
        if pd.isna(row.get('Unidade', None)):
            descricao_atualizada, unidade = extrair_unidade(row['descrição'])
            metadata_df.loc[index, 'descrição'] = descricao_atualizada
            metadata_df.loc[index, 'Unidade'] = unidade

        # Atualizar timestamp do momento da verificação
        metadata_df.loc[index, 'atualizado_em'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Imprimir os detalhes da série em uma única linha
        print(f"Código: {codigo_serie}, Nome: {row['nome'][:50]}", end="")  # Limita o nome para 50 caracteres para manter a linha compacta

        metadados, status = atualizar_metadados(codigo_serie)
        
        # Atualizar o status, independentemente de sucesso ou erro
        metadata_df.loc[index, 'status_download'] = status
        
        if metadados:
            metadata_df.loc[index, 'num_registros'] = metadados['num_registros']
            metadata_df.loc[index, 'primeira_data'] = metadados['primeira_data']
            metadata_df.loc[index, 'ultima_data'] = metadados['ultima_data']
            metadata_df.loc[index, 'intervalo_dias'] = metadados['intervalo_dias']
            metadata_df.loc[index, 'valor_min'] = metadados['valor_min']
            metadata_df.loc[index, 'valor_max'] = metadados['valor_max']
            metadata_df.loc[index, 'media_valor'] = metadados['media_valor']
            metadata_df.loc[index, 'desvio_padrao_valor'] = metadados['desvio_padrao_valor']
            metadata_df.loc[index, 'media_frequencia'] = metadados['media_frequencia']
            metadata_df.loc[index, 'frequencia'] = metadados['frequencia']
            metadata_df.loc[index, 'status_atividade'] = metadados['status_atividade']
            
            # Completar a linha com as datas e o status
            print(f", de: {metadados['primeira_data']}, a: {metadados['ultima_data']}, Registros: {metadados['num_registros']}, Status: {status}")
        else:
            # Se houve erro, mostrar o status de falha
            print(f", Registros: 0, Status: {status}")
        
        # Salvar o arquivo atualizado a cada iteração, mesmo que tenha ocorrido erro
        metadata_df.to_csv(output_file, index=False)

# Exibir a mensagem final
print("Processo concluído e metadados atualizados.")


Código: 10419, Nome: FDI - United Kingdom - Manufacture of metal produc, Registros: 0, Status: Sucesso
Código: 10789, Nome: FDI - Uruguay - Manufacture of medical, precision , Registros: 0, Status: Sucesso
Código: 10838, Nome: FDI - Bermuda - Manufacture of metal products, Registros: 0, Status: Sucesso
Código: 10854, Nome: FDI - Bermuda - Manufacture of metal products, Registros: 0, Status: Sucesso
Código: 10859, Nome: FDI - Panama - Manufacture of cellulose, paper and, Registros: 0, Status: Erro de requisição: Expecting value: line 1 column 1 (char 0)
Código: 10861, Nome: FDI - Bermuda - Manufacture of medical, precision , Registros: 0, Status: Sucesso
Código: 10865, Nome: FDI - Chile - Manufacture of non-metallic mineral , Registros: 0, Status: Sucesso
Código: 10869, Nome: FDI - Gibraltar - Manufacture of wood products, Registros: 0, Status: Sucesso
Código: 10879, Nome: FDI - Singapore - Manufacture of cellulose, paper , Registros: 0, Status: Sucesso
Código: 10880, Nome: FDI - Singap

In [9]:
import pandas as pd

# Caminho do arquivo CSV atualizado
file_path = 'BCB_metadata_atualizado.csv'

# Carregar o arquivo de metadados
metadata_df = pd.read_csv(file_path, sep=',', quotechar='"', on_bad_lines='skip')

# Filtrar registros que tiveram sucesso no download e estão ativos
# Considera-se que o status do download foi "Sucesso" e o status de atividade é "Ativa"
ativos_df = metadata_df[
    #(metadata_df['status_download'] == 'Sucesso') &
    (metadata_df['status_atividade'] == 'Ativa')
]

# Ordenar os registros filtrados pelo campo 'código' como inteiro
ativos_df = ativos_df.sort_values(by='código')

# Exibir os registros filtrados
if not ativos_df.empty:
    print("Registros ativos e com download bem-sucedido:")
else:
    print("Nenhum registro ativo encontrado com download bem-sucedido.")


Registros ativos e com download bem-sucedido:


In [10]:
ativos_df

,código,nome,descrição,atualizado_em,Unidade,num_registros,primeira_data,ultima_data,intervalo_dias,valor_min,valor_max,media_valor,desvio_padrao_valor,media_frequencia,frequencia,status_atividade,status_download
0,1,"Exchange rate, Free, United States dollar (sal...","Exchange rate, Free, United States dollar (sal...",2024-09-15 15:54:06,c.m.u./US$,9972.0,1984-11-28,2024-09-13,2.0,8.290000e-01,7.115300e+04,8.484459e+02,4.693352e+03,1.457481,Diária,Ativa,NaN
916,11,"Interest rate, Selic","Interest rate, Selic.",2024-09-15 16:11:42,% p.d.,9597.0,1986-06-04,2024-09-13,2.0,0.000000e+00,3.626000e+00,2.589656e-01,4.751020e-01,1.456809,Diária,Ativa,NaN
1882,12,"Interest rate, CDI","Interest rate, CDI.",2024-09-15 16:32:27,% p.d.,9638.0,1986-03-06,2024-09-12,3.0,7.469000e-03,3.970667e+00,2.631420e-01,4.818885e-01,1.459846,Diária,Ativa,NaN
5702,23,"Daily savings deposits balance, Brazilian syst...","Daily savings deposits balance, Brazilian syst...",2024-09-15 17:40:18,c.m.u. (thousand),8275.0,1991-08-22,2024-08-15,31.0,3.294983e+07,1.579828e+12,1.651508e+10,1.111415e+11,1.455831,Diária,Ativa,NaN
5810,24,"Daily saving deposits net inflow, Brazilian sy...","Daily saving deposits net inflow, Brazilian sy...",2024-09-15 17:41:58,c.m.u. (thousand),8275.0,1991-08-22,2024-08-15,31.0,-4.012215e+09,5.833764e+09,2.019914e+07,2.742354e+08,1.455831,Diária,Ativa,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4670,20359,"Foreign exchange interbank spot volume, USD, T+1","Foreign exchange interbank spot volume, USD, T+1.",2024-09-15 17:22:22,US$,7581.0,1994-07-04,2024-09-13,2.0,0.000000e+00,7.728700e+09,4.543453e+08,5.904142e+08,1.454821,Diária,Ativa,NaN
4722,20479,"Claims on Other Sectors, Monetary Authority","Claims on Other Sectors, Monetary Authority.",2024-09-15 17:23:30,c.m.u. (million),237.0,2004-12-01,2024-08-01,45.0,0.000000e+00,1.170000e+03,3.563291e+01,1.824131e+02,30.308017,Mensal,Ativa,NaN
4725,20499,"Assets, Monetary Authority","Assets, Monetary Authority.",2024-09-15 17:23:33,c.m.u. (million),237.0,2004-12-01,2024-08-01,45.0,1.949230e+05,3.046324e+06,1.406689e+06,7.871805e+05,30.308017,Mensal,Ativa,NaN
4727,20500,"Liability, Monetary Authority","Liability, Monetary Authority.",2024-09-15 17:23:35,c.m.u. (million),237.0,2004-12-01,2024-08-01,45.0,1.949230e+05,3.046324e+06,1.406689e+06,7.871805e+05,30.308017,Mensal,Ativa,NaN


In [11]:
# Salvar os registros filtrados em um novo arquivo, se necessário
ativos_df.to_csv('BCB_metadata_active.csv', index=False)  # Caso queira salvar os ativos